Download pre-trained Word2Vec model


In [ ]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

corpus = api.load('text8')
model = Word2Vec(corpus)

Getting embedding of a sentence

In [ ]:
import numpy as np

def get_sentence_embedding(model, text):
  # This method takes in the trained model and the input sentence
  # and returns the embedding of the sentence as the average embedding
  # of its words
  words = text.split(" ")
  count = 0
  for i in range(1, len(words)):
    try:
      if count == 0:
        vector = model.wv[words[i]]
      else: 
        vector = np.copy(vector+model.wv[words[i]])
      count+=1
    except:
      continue
  return vector/count

# Sample code to extract vector for a sentence
get_sentence_embedding(model, "test text embedding")

Reading TSV file and saving embeddings

In [ ]:
import pandas as pd

def read_tweets_get_vectors(tweet_file_path):
  # This method takes in the file path for the twitter file, and return a
  # dicationary of dictionaries. In the first dictionary the keys are the
  # tweet labels (3 classes), and the values are another dictionary with
  # tweet id as the key and values are tuple of (vector, tweet text)
    df = pd.read_csv(tweet_file_path, sep=',', header=0)
    dic_result = {}
    df1 = df[['tweet_id', 'text', 'airline_sentiment']]
    for index in range(len(df1)):
        try:
            vetor_rep = get_sentence_embedding(model, df.loc[index, "text"].lower())
            label = df.loc[index, "airline_sentiment"]
            tweet_id = df.loc[index, "tweet_id"]
            if label in dic_result:
                dic_result[label][tweet_id] = (vetor_rep, df.loc[index, "text"].lower())
            else:
                dic_result[label] = {tweet_id: (vetor_rep, df.loc[index, "text"].lower())}
        except:
            pass
    return dic_result

twitter_data = read_tweets_get_vectors("Tweets.csv")
for key in twitter_data.keys():
  print(key + "\t\t number of instances: " + str(len(twitter_data[key])))

Code to generate training, validation, and test sets

In [ ]:
import random

def split_data(twitter_data):
  # takes in the dictionary from the previous step and generate
  # the training, validation, and test sets. Note that the labels 
  # are represented as one-hot codings.
    training_x = []
    training_y = []

    validation_x = []
    validation_y = []

    test_x = []
    test_y = []

    for label in twitter_data:

        # labels are indicated as one hot coding [negative, neutral, positive]
        if label == "negative":
            n_label = [1, 0, 0]
        elif label == "neutral":
            n_label = [0, 1, 0]
        else:
            n_label = [0, 0, 1]
        temp_dic = twitter_data[label]
        lst_tweet_ids = list(temp_dic.keys())
        #### Splitting by 80-10-10
        ## Note that you could alternatively use sklearn split method
        train_length = int(len(lst_tweet_ids)*0.8)
        train_ids = lst_tweet_ids[ :train_length]
        remaining = lst_tweet_ids[train_length:]
        test_lenght = int(len(remaining)*0.5)
        test_ids = remaining[:test_lenght]
        validation_id = remaining[test_lenght:]

        for tweet_id in train_ids:
            training_x.append(temp_dic[tweet_id][0])
            training_y.append(n_label)
        for tweet_id in validation_id:
            validation_x.append(temp_dic[tweet_id][0])
            validation_y.append(n_label)
        for tweet_id in test_ids:
            test_x.append(temp_dic[tweet_id][0])
            test_y.append(n_label)

    # The reason we apply this shuffling is to make sure 
    # when passing batches to the network, we see different items 
    c = list(zip(training_x, training_y))
    random.shuffle(c)
    training_x, training_y = zip(*c)

    return training_x, training_y, validation_x, validation_y, test_x, test_y

# Sample usage
training_x, training_y, validation_x, validation_y, test_x, test_y = split_data(twitter_data)

Here goes your code for your Feedfoward network Design

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch


class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, hidden_dim_2, hidden_dim_3, p, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Define layers


    def forward(self, x):
        # Your network forward pass

        # modify this line
        return None

Training the network
Define a class, with properties such as size of hidden layers
loss function, optimizer, training method, test method, and accuracy

In [ ]:
from tqdm import tqdm

class ModelModule():
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # here goes your parameters

  # sample code to define your model 
  # model = FeedforwardNeuralNetModel(input_dim, hidden_dim_1, out_dim)
  # model.to(devide)


  def calculate_accuracy(self, y_true, y_pred):
    # this method will be used to calculate the accuracy of your model
      correct = (y_true.argmax (dim = 1) == y_pred.argmax (dim = 1)).float()
      acc = correct.sum() / len(correct)
      return acc

  def training(self, tfidfX_train, Y_train, tfidfX_val, Y_val, num_epochs):
    # this method will be used for training your model
    # inputs are the training and validation sets

    # You can define batch size of your choice
    batch_size = 20
    X_train_mini_batches = torch.split(tfidfX_train, batch_size)
    Y_train_mini_batches = torch.split(Y_train, batch_size)

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    best_accuracy = 0

    for epoch in tqdm(range(num_epochs)):
        epoch_loss = 0
        epoch_accuracy = 0
        for X_train_mini_batch, Y_train_mini_batch in zip(X_train_mini_batches, Y_train_mini_batches):
            X_train_mini_batch = X_train_mini_batch.to(self.device)
            Y_train_mini_batch = Y_train_mini_batch.to(self.device)
            # Continue code here to train the network
        # here check your validation set
        # you have to save the model with the best loss or maybe accuracy?
  
